In [ ]:
%config InlineBackend.figure_format = 'retina'
%pylab inline
%load_ext autoreload
%autoreload 2

import scipy.io
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
import torbspikes as ts
from keras.utils.visualize_util import plot as kplot
#kplot(mod)

from IPython.display import SVG
from keras.utils.visualize_util import model_to_dot
import xgboost as xgb

from sklearn.metrics import accuracy_score

indir = "/Users/Torben/Documents/Kording/GLMDeep/M1 Stevenson Binned.mat"
outdir="/Users/Torben/Documents/Kording/GLMDeep/results_9.28.16/"


In [ ]:
# get data
monkey_dat = scipy.io.loadmat(indir)
monkey_dat.keys()
all_spikes = monkey_dat['spikes']

In [ ]:
neuron_n = 193
kernel_params = {"kernel_size":[3,5,10,15],"kernel_type":["cos", "cos","cos","cos"],"X":True,"y":False}
X_train,X_test,y_train,y_test = ts.bookkeeping.organize_data(all_spikes=all_spikes,my_neuron=neuron_n,
                                              subsample=None,train_test_ratio=0.9,
                                              winsize=5,convolve_params=None,
                                              RNN_out=False,verbose=False)

In [ ]:
X_train_rnn,X_test_rnn,y_train_rnn,y_test_rnn = ts.bookkeeping.organize_data(all_spikes=all_spikes,my_neuron=neuron_n,
                                              subsample=None,train_test_ratio=0.9,
                                              winsize=5,convolve_params=None,
                                              RNN_out=True,verbose=False)

In [ ]:
glm_pred,glm_model = ts.models.GLM_poisson(X_train,y_train,X_test,return_model=True)

In [ ]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.core import Lambda
from keras.layers.recurrent import LSTM
from keras.regularizers import l1, activity_l1

def RNN_into_GLM(Xr,Yr,Xt):    

    model = Sequential()
    model.add(LSTM(10, input_dim=196))
    model.add(Dense(1, input_dim=10, init='uniform', activation='linear'))
    model.add(Lambda(lambda x: np.exp(x)))
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.fit(Xr, Yr, nb_epoch=10, batch_size=16, verbose=2)
    result = model.predict(Xt)
    return result, model


In [ ]:
X_train_rnn_shrink,X_test_rnn_shrink,y_train_rnn_shrink,y_test_rnn_shrink = ts.bookkeeping.organize_data(all_spikes=all_spikes,my_neuron=neuron_n,
                                              subsample=None,train_test_ratio=0.9,
                                              winsize=5,convolve_params=None,
                                              RNN_out=True,verbose=False,shrink_X=0.1)

In [ ]:

'''    
glm_r2 = []
rnn_r2 = []
for neuron_n in range(50):

    X_train,X_test,y_train,y_test = ts.bookkeeping.organize_data(all_spikes=all_spikes,my_neuron=neuron_n,
                                                  subsample=None,train_test_ratio=0.9,
                                                  winsize=5,convolve_params=None,
                                                  RNN_out=False,verbose=False)
    glm_pred,glm_model = ts.models.GLM_poisson(X_train,y_train,X_test,return_model=True)
    
    X_train_rnn_shrink,X_test_rnn_shrink,y_train_rnn,y_test_rnn = ts.bookkeeping.organize_data(all_spikes=all_spikes,my_neuron=neuron_n,
                                                  subsample=None,train_test_ratio=0.9,
                                                  winsize=5,convolve_params=None,
                                                  RNN_out=True,verbose=False,shrink_X=0.1)
    rnn_pred, rnn_model = RNN_into_GLM(X_train_rnn_shrink,y_train_rnn,X_test_rnn_shrink)
    
    glm_r2.append(ts.math.poisson_pseudoR2(y_test,glm_pred,verbose=False))
    rnn_r2.append(ts.math.poisson_pseudoR2(y_test_rnn,rnn_pred,verbose=False))
np.save('glm1to50',glm_r2)
np.save('rnn1to50',rnn_r2)
'''

In [ ]:
glm_r2 = np.load('/Users/Torben/Documents/Kording/GLMDeep/glm1to50.npy')
rnn_r2 = np.load('/Users/Torben/Documents/Kording/GLMDeep/rnn1to50.npy')

new_inds = ts.bookkeeping.sort_spikes(all_spikes, method='sum')
#plt.plot(new_inds)
#ts.plotting.plot_preds(y_test_rnn,rnn_pred)
glm_r2_sorted_by_sc = []
rnn_r2_sorted_by_sc = []
#inds_sorted_by_sc = []
for i in new_inds:
    if i in range(50):
        glm_r2_sorted_by_sc.append(glm_r2[i])
        rnn_r2_sorted_by_sc.append(rnn_r2[i])

plt.plot(glm_r2_sorted_by_sc)
plt.plot(rnn_r2_sorted_by_sc,'r')

In [ ]:
high_spiking = ts.bookkeeping.sort_spikes(all_spikes,method='sum')[-10:]
mid_spiking = ts.bookkeeping.sort_spikes(all_spikes,method='sum')[100:110]
low_spiking = ts.bookkeeping.sort_spikes(all_spikes,method='sum')[:10]

glm_low = []
rnn_low = []

for neuron_n in low_spiking:

    X_train,X_test,y_train,y_test = ts.bookkeeping.organize_data(all_spikes=all_spikes,my_neuron=neuron_n,
                                                  subsample=None,train_test_ratio=0.9,
                                                  winsize=5,convolve_params=None,
                                                  RNN_out=False,verbose=False)
    glm_pred,glm_model = ts.models.GLM_poisson(X_train,y_train,X_test,return_model=True)
    
    X_train_rnn_shrink,X_test_rnn_shrink,y_train_rnn,y_test_rnn = ts.bookkeeping.organize_data(all_spikes=all_spikes,my_neuron=neuron_n,
                                                  subsample=None,train_test_ratio=0.9,
                                                  winsize=5,convolve_params=None,
                                                  RNN_out=True,verbose=False,shrink_X=0.1)
    rnn_pred, rnn_model = RNN_into_GLM(X_train_rnn_shrink,y_train_rnn,X_test_rnn_shrink)
    
    glm_low.append(ts.math.poisson_pseudoR2(y_test,glm_pred,verbose=False))
    rnn_low.append(ts.math.poisson_pseudoR2(y_test_rnn,rnn_pred,verbose=False))

glm_mid = []
rnn_mid = []

for neuron_n in mid_spiking:

    X_train,X_test,y_train,y_test = ts.bookkeeping.organize_data(all_spikes=all_spikes,my_neuron=neuron_n,
                                                  subsample=None,train_test_ratio=0.9,
                                                  winsize=5,convolve_params=None,
                                                  RNN_out=False,verbose=False)
    glm_pred,glm_model = ts.models.GLM_poisson(X_train,y_train,X_test,return_model=True)
    
    X_train_rnn_shrink,X_test_rnn_shrink,y_train_rnn,y_test_rnn = ts.bookkeeping.organize_data(all_spikes=all_spikes,my_neuron=neuron_n,
                                                  subsample=None,train_test_ratio=0.9,
                                                  winsize=5,convolve_params=None,
                                                  RNN_out=True,verbose=False,shrink_X=0.1)
    rnn_pred, rnn_model = RNN_into_GLM(X_train_rnn_shrink,y_train_rnn,X_test_rnn_shrink)
    
    glm_mid.append(ts.math.poisson_pseudoR2(y_test,glm_pred,verbose=False))
    rnn_mid.append(ts.math.poisson_pseudoR2(y_test_rnn,rnn_pred,verbose=False))

glm_high = []
rnn_high = []

for neuron_n in high_spiking:

    X_train,X_test,y_train,y_test = ts.bookkeeping.organize_data(all_spikes=all_spikes,my_neuron=neuron_n,
                                                  subsample=None,train_test_ratio=0.9,
                                                  winsize=5,convolve_params=None,
                                                  RNN_out=False,verbose=False)
    glm_pred,glm_model = ts.models.GLM_poisson(X_train,y_train,X_test,return_model=True)
    
    X_train_rnn_shrink,X_test_rnn_shrink,y_train_rnn,y_test_rnn = ts.bookkeeping.organize_data(all_spikes=all_spikes,my_neuron=neuron_n,
                                                  subsample=None,train_test_ratio=0.9,
                                                  winsize=5,convolve_params=None,
                                                  RNN_out=True,verbose=False,shrink_X=0.1)
    rnn_pred, rnn_model = RNN_into_GLM(X_train_rnn_shrink,y_train_rnn,X_test_rnn_shrink)
    
    glm_high.append(ts.math.poisson_pseudoR2(y_test,glm_pred,verbose=False))
    rnn_high.append(ts.math.poisson_pseudoR2(y_test_rnn,rnn_pred,verbose=False))
np.save('glmlow',glm_low)
np.save('rnnlow',rnn_low)
np.save('glmmid',glm_mid)
np.save('rnnmid',rnn_mid)
np.save('glmhigh',glm_high)
np.save('rnnhigh',rnn_high)

    

In [ ]:
#print glm_mid
#rnn_mid
plt.violinplot([glm_mid,rnn_mid],[1,2])
plt.violinplot([glm_high,rnn_high],[4,5])

In [12]:
avg_neuron = ts.bookkeeping.sort_spikes(all_spikes,method='sum')[all_spikes.shape[0]/2]
these_neurons = [avg_neuron]

for neuron_n in these_neurons:
    X_train,X_test,y_train,y_test = ts.bookkeeping.organize_data(all_spikes=all_spikes,my_neuron=neuron_n,
                                                  subsample=None,train_test_ratio=0.7,
                                                  winsize=5,convolve_params=None,
                                                  RNN_out=False,verbose=False)
    
    glm_pred = ts.models.GLM_poisson(X_train,y_train,X_test,return_model=False)
    nn_pred = ts.models.NN_poisson(X_train,y_train,X_test,return_model=False)
    xgb_pred = ts.models.XGB_poisson(X_train,y_train,X_test)

    X_train_rnn_shrink,X_test_rnn_shrink,y_train_rnn,y_test_rnn = ts.bookkeeping.organize_data(all_spikes=all_spikes,my_neuron=neuron_n,
                                                  subsample=None,train_test_ratio=0.7,
                                                  winsize=5,convolve_params=None,
                                                  RNN_out=True,verbose=False,shrink_X=0.1)
    rnn_pred, rnn_model = ts.models.RNN_poisson(X_train_rnn_shrink,y_train_rnn,X_test_rnn_shrink,return_model=True)

    
    _, glm_accuracy = ts.models.XGB_ensemble(glm_pred,y_test)
    _, nn_accuracy = ts.models.XGB_ensemble(nn_pred,y_test)
    _, rnn_accuracy = ts.models.XGB_ensemble(rnn_pred,y_test)
    _, xgb_accuracy = ts.models.XGB_ensemble(xgb_pred,y_test)
    
    print "GLM PseudoR2: " + str(ts.math.poisson_pseudoR2(y_test,glm_pred))
    print "NN PseudoR2: " + str(ts.math.poisson_pseudoR2(y_test,nn_pred))
    print "RNN PseudoR2: " + str(ts.math.poisson_pseudoR2(y_test,rnn_pred))
    print "XGB PseudoR2: " + str(ts.math.poisson_pseudoR2(y_test,xgb_pred))

    print("GLM Accuracy: %.2f%%" % (glm_accuracy * 100.0))
    print("NN Accuracy: %.2f%%" % (nn_accuracy * 100.0))
    print("RNN Accuracy: %.2f%%" % (rnn_accuracy * 100.0))
    print("XGB Accuracy: %.2f%%" % (xgb_accuracy * 100.0))
    
    #use ensmble methods to predict spiking
    ensemble_train_full = np.squeeze(np.array([glm_pred,nn_pred,rnn_pred,xgb_pred])).T

    ensemble_preds, accuracy = ts.models.XGB_ensemble(ensemble_train_full,y_test)

    print("Ensemble Accuracy: %.2f%%" % (accuracy * 100.0))

using only data preceeding spikes with window_size: 5
Epoch 1/3
10871/10871 [==============================] - 0s - loss: 0.9346     
Epoch 2/3
10871/10871 [==============================] - 0s - loss: 0.7850     
Epoch 3/3
10871/10871 [==============================] - 0s - loss: 0.7642     
using only data preceeding spikes with window_size: 5
Epoch 1/10
1s - loss: 0.3957
Epoch 2/10
1s - loss: 0.3721
Epoch 3/10
1s - loss: 0.3715
Epoch 4/10
1s - loss: 0.3705
Epoch 5/10
1s - loss: 0.3692
Epoch 6/10
1s - loss: 0.3670
Epoch 7/10
1s - loss: 0.3627
Epoch 8/10
1s - loss: 0.3536
Epoch 9/10
1s - loss: 0.3384
Epoch 10/10
1s - loss: 0.3297


ValueError: too many values to unpack

In [ ]:
#ensemble_validate_test,ensemble_glm_pred, glm_accuracy = ts.models.XGB_ensemble(glm_pred,y_test)


In [ ]:
validate_train = np.squeeze(np.array([glm_pred[cross_validation_ind:],cnn_pred[cross_validation_ind:],rnn_pred[cross_validation_ind:]])).T
validate_test = y_test[cross_validation_ind:]
print ensemble_validate.shape
y_pred = model.predict(validate_train)

In [ ]:

model.fit(ensemble_train, ensemble_test)
ensemble_pred = model.predict(ensemble_validate_train)

#calculate accuracy

accuracy = accuracy_score(ensemble_validate_test,ensemble_pred)

print("Accuracy: %.2f%%" % (accuracy * 100.0))


In [ ]:
plt.hist(y_pred)

In [ ]:
#from keras.utils.visualize_util import plot as kplot
#kplot(mod)

#from IPython.display import SVG
#from keras.utils.visualize_util import model_to_dot

SVG(model_to_dot(rnn_model,show_shapes=True,show_layer_names=False).create(prog='dot', format='svg'))
#SVG(model_to_dot(glm_model,show_shapes=True,show_layer_names=False).create(prog='dot', format='svg'))
#model_to_dot??

In [ ]:
t = np.array((1,2,3,np.nan))
any(np.isnan(t))

In [ ]:
t = range(10)
t_z = ((t-np.mean(t))/np.std(t))
t_z = t_z-min(t_z)
print t_z

In [ ]:
print '\n'
print 'lax'